<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/Cryptography/CryptoWorkout_Week15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 타원곡선 암호

## 타원곡선의 덧셈 규칙

타원곡선 $𝐸:𝑦^2=𝑥^3+𝑏𝑥+𝑐$ 위의 두 점
$𝑃_1=(𝑥_1, 𝑦_1 ), 𝑃_2=(𝑥_2,𝑦_2 )$에 대해,  
$𝑃_1+𝑃_2=𝑃_3=(𝑥_3,𝑦_3)$라 하면 
>>
$$𝑥_3=𝑚^2−𝑥_1−𝑥_2$$
$$𝑦_3=𝑚(𝑥_1−𝑥_3 )−𝑦_1$$
여기서 
$𝑚=
\begin{cases}
(𝑦_2−𝑦_1)/(𝑥_2−𝑥_1 ),&   \text{𝑖𝑓  } 𝑃_1≠𝑃_2 \\
(3𝑥_1^2+𝑏)/(2𝑦_1 ),&  \text{𝑖𝑓  } 𝑃_1=𝑃_2 
\end{cases}$


##타원곡선 위의 점의 갯수
Hasse의 정리: $𝑁$을 $𝐸 \pmod{p}$의 점의 개수라 하면, 
$$|𝑁−𝑝−1|<2\sqrt{𝑝}$$


## 실습문제 

1. 다음을 풀어라.
  1. 타원곡선 $E: y^2 \equiv x^3 -2 \pmod{7}$의 점들을 모두 나열하여라.
  2. $E$위에서 $(3,2)+(5,5)$를 계산하여라.
  3. $E$위에서 $(3,2)+(3,2)$를 계산하여라.

2. 소수 $p$에 대해, $E$가 법 $p$위의 타원곡선 그리고 $P$가 그 위의 한 점이라 하자.
  1. $E$위에는 유한개의 점들만 있음을 보여라. (따라서, $P$는 유한개의 서로 다른 배수점-$mP$꼴의 점들-이 있다)
  2. $iP=jP$이고 $i>j$이면 $(i-j)P=\infty$임을 보여라.
  
3.  $P=(2,3)$이 타원곡선 $y^2\equiv x^3-10x+21 \pmod{557}$위의 점이다.
  0. [Python] 5주차에서 완성했던 법 $n$에 대한 빠른 거듭제곱 명령어 `Modular_power(a, m, n)`를 참조하여 타원곡선에서 점 $P$의 배수점 $mP$를 빠르게 구하는 명령어 `MultipleP(P, m)`을 완성하여라.
  2. [Python] 명령어 `MultipleP(P, m)`와 2번을 참고하여  $189P = \infty$이지만 $63P \ne \infty, 27P \ne \infty$임을 보여라.
  3. $P$의 위수가 189임을 보여라.
  4. Hasse의 정리와 대수학의 Lagrange 정리를 이용하여 이 타원곡선 위의 점이 총 567개 임을 보여라.


## 타원곡선 위의 점의 배수점 계산하기

주어진 타원곡선 $E:y^2 \equiv x^3+bx+c \pmod{p}$, 그 위의 점 $P=(x_1, y_1)$ 그리고 자연수 $m>1$에 대해 $mP$ 계산하는 코드를 작성해보자. 
$\infty=(\infty, \infty)$는 편의상 $('inf', 'inf')$로 표시한다.

먼저, 

1. 두 점 $P1, P2$의 덧셈을 구해주는 명령어 `SumP(P1, P2)`을 작성하고
2. `m`을 2진법으로 바꿔주는 명령어 `Num_to_Binary(m)`를 이용하여
3. 이를 이용하여 $mP$를 구해주는 명령어 `MultipleP(P, m)`을 완성해보자.

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving Week15_0.jpg to Week15_0.jpg
User uploaded file "Week15_0.jpg" with length 3334459 bytes


In [ ]:
# 5주차: 2진법 변환 명령어와 법 n에 대해 a의 m제곱 빠른 계산 명령어

def Num_to_Binary(m):
  l = []
  q = m
  while q > 0:
    r = q % 2
    q = q // 2
    l = l+[r]
  return l

def egcd(a, b):
    if a == 0:
      if b>0:
        return (b, 0, 1)
      else:
        return (-b, 0, -1)
    
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def Modular_power(a, m, n):
  l = Num_to_Binary(m)[::-1]
  x = 1
  for i in l:
    x = (x**2) % n
    if i==1:
      x = (x*a) % n
  return x

In [ ]:
Num_to_Binary(64)

[0, 0, 0, 0, 0, 0, 1]

$E:y^2 \equiv x^3+bx+c \pmod{p}$

In [ ]:
p = 557
b = -10
c = 21

def SumP(P1, P2):
#  print(P1, P2)
  if P1==('inf', 'inf'):
    return P2
  elif P2==('inf', 'inf'):
    return P1
  elif P1==P2:
    if (2*P1[1]) % p == 0: # m값이 무한대이면 덧셈은 무한대점
      return 'inf', 'inf'
    else:
      m = ((3*(P1[0]**2)+b)*modinv(2*P1[1], p)) % p
  else:
    if (P2[0]-P1[0]) % p == 0: # m값이 무한대이면 덧셈은 무한대점
      return 'inf', 'inf'
    else:
      m = ((P2[1]-P1[1])*modinv(P2[0]-P1[0], p)) % p
   
  x3 = (m**2 - P1[0] - P2[0]) % p
  y3 = (m*(P1[0]-x3) - P1[1]) % p 

  return x3, y3

def MultipleP(P, m):
  if m < 1:
    return FALSE
  else:
    l = Num_to_Binary(m)[::-1]
    Q = ('inf', 'inf') # 이 점이 덧셈에 대한 항등원 입니다.
    for i in l:
        Q = SumP(Q,Q)
        if i==1:
            Q =  SumP(P,Q)       

    
    return Q


In [ ]:
MultipleP((2, 3), 190)

(2, 3)

In [ ]:
MultipleP((2, 3), 1)    #63 , 27
 

(2, 3)

In [ ]:
MultipleP((2, 3), 189)


('inf', 'inf')

In [ ]:
#MultipleP((2, 3), 190) = MultipleP((2, 3), 1) 이므로 (190-1)*P = 189*P = ('inf', 'inf')
MultipleP((2, 3), 189)

('inf', 'inf')

In [ ]:
MultipleP((2, 3), 63)


(38, 535)

In [ ]:
MultipleP((2, 3), 27)


(136, 360)